# Reservoir Operations under Uncertainty

## Import necessary libraries

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats as ss
import statsmodels.api as sm
from matplotlib import pyplot as plt
import matplotlib
%matplotlib inline

## Define simulation parameters and plotting function

In [ ]:
zCrit025 = ss.norm.ppf(0.025)
zCrit975 = ss.norm.ppf(0.975)

rho = 0.5
logAvg = np.array([2.9, 3.9, 4.6, 4.6, 4.1, 3.7, 3.3, 2.9, 2.6, 2.3, 2.2, 2.2])
logSd = np.array([0.5, 0.4, 0.3, 0.4, 0.3, 0.2, 0.4, 0.3, 0.3, 0.3, 0.3, 0.3])

def plotForecast(i, S, K, Qfcst, Qfcst025, Qfcst975, Qsim=np.zeros(6)):
    fig = plt.figure()
    ax = fig.add_subplot(1,2,1)
    if i != 0:
        ax.scatter(range(1,i+1),Qsim[0:i],color="#e6550d")
        ax.plot(range(1,i+1),Qsim[0:i],color="#e6550d")
        ax.plot(range(i,7),Qfcst[(i-1)::],color="#3182bd")
        ax.fill_between(range(i,7),Qfcst025[(i-1)::],Qfcst975[(i-1)::],
                        color="#9ecae1")
    else:
        ax.plot(range(1,7),Qfcst,color="#3182bd")
        ax.fill_between(range(1,7),Qfcst025,Qfcst975,color="#9ecae1")
    ax.set_ylim([0,250])
    ax.set_ylabel("Forecasted inflow")
    ax.set_xlabel("t")
    ax.set_title("Observations up to\n and Forecast after t=" + str(i))
    
    ax = fig.add_subplot(1,2,2)
    ax.scatter(range(i+1),S[0:(i+1)],color="#31a354")
    ax.plot(range(i+1),S[0:(i+1)],color="#31a354")
    ax.plot(range(7),np.repeat(K,7),color="k")
    ax.set_ylim([0,250])
    ax.set_ylabel("Storage")
    ax.set_xlabel("t")
    ax.set_title("Storage up to t=" + str(i))
    
    fig.tight_layout()
    
    plt.show()

## Run 6-period simulation, starting with a full reservoir. The goal is to have the highest reservoir level at the end of the simulation without overtopping the dam.

In [ ]:
# capacity of reservoir
K = 200

# maximum release
Rmax = 150

# vector of storage values; start with full reservoir
S = np.empty(7)
S[0] = K

print("Current storage: " + str(S[0]))
print("Reservoir capacity: " + str(K))

# get release decision from user
R = np.empty([6])

# sample initial flow
Zfcst = np.empty([6])
Qfcst = np.empty([6])
Qfcst025 = np.empty([6])
Qfcst975 = np.empty([6])

Zfcst[0] = ss.norm.ppf(0.5) # median for first time step
Qfcst[0] = int(np.exp(Zfcst[0]*logSd[0] + logAvg[0]))
Qfcst025[0] = int(np.exp((Zfcst[0]+zCrit025)*logSd[0] + logAvg[0]))
Qfcst975[0] = int(np.exp((Zfcst[0]+zCrit975)*logSd[0] + logAvg[0]))
# forecast the next 5 time steps
for i in range(5):
    Zfcst[i+1] = rho*(Zfcst[i])
    Qfcst[i+1] = int(np.exp(Zfcst[i+1]*logSd[i+1] + logAvg[i+1]))
    Qfcst025[i+1] = int(np.exp((Zfcst[i+1]+zCrit025)*logSd[i+1] + logAvg[i+1]))
    Qfcst975[i+1] = int(np.exp((Zfcst[i+1]+zCrit975)*logSd[i+1] + logAvg[i+1]))
    
index = []
for i in range(6):
    index.append("t+" + str(i+1))
    
df = pd.DataFrame({"Average Fcst": Qfcst, "2.5%-ile": Qfcst025, 
                   "97.5%-ile": Qfcst975}, index = index)
    
print(df)

plotForecast(0, S, K, Qfcst, Qfcst025, Qfcst975)

while True:
    R[0] = input("How much would you like to release? (max release = " 
     + str(Rmax) + ")\n")
    if R[0] > Rmax:
        print("Release must be < " + str(Rmax) + "\n")
    elif R[0] < 0:
        print("Release must be >= 0\n")
    else:
        break

Zsim = np.empty([6])
Zsim[0] = ss.norm.rvs(0,1,1)
Qsim = np.empty([6])
Qsim[0] = int(np.exp(Zsim[0]*logSd[0] + logAvg[0]))

print("Inflow: " + str(Qsim[0]))
print("Release: " + str(min(R[0], S[0] + Qsim[0])))

# simulate next storage value
S[1] = S[0] + Qsim[0] - min(R[0], S[0] + Qsim[0])
if S[1] > K:
    print("Oh no! You overtopped the dam")
else:
    # generate next 5 time steps
    for i in range(1,6):
        print("Current Storage: " + str(S[i]))
        print("Reservoir capacity: " + str(K))
        # update forecast
        Zfcst[i-1] = Zsim[i-1]
        Qfcst[i-1] = Qsim[i-1]
        Qfcst025[i-1] = Qsim[i-1]
        Qfcst975[i-1] = Qsim[i-1]
        # update forecast for next 5-i time steps
        for j in range(i,6):
            Zfcst[j] = rho*(Zfcst[j-1])
            Qfcst[j] = int(np.exp(Zfcst[j]*logSd[j] + logAvg[j]))
            Qfcst025[j] = int(np.exp((Zfcst[j]+zCrit025)*logSd[j] + logAvg[j]))
            Qfcst975[j] = int(np.exp((Zfcst[j]+zCrit975)*logSd[j] + logAvg[j]))
            
        index = []
        for j in range(6-i):
            index.append("t+" + str(j+1))
    
        df = pd.DataFrame({"Average Fcst": Qfcst[i::], "2.5%-ile": Qfcst025[i::], 
                   "97.5%-ile": Qfcst975[i::]}, index = index)
    
        print(df)
        
        plotForecast(i, S, K, Qfcst, Qfcst025, Qfcst975, Qsim)
                        
        # get release decision from user
        while True:
            R[i] = input("How much would you like to release? (max release = " 
             + str(Rmax) + ")\n")
            if R[i] > Rmax:
                print("Release must be < " + str(Rmax) + "\n")
            elif R[i] < 0:
                print("Release must be >= 0\n")
            else:
                break
            
        # generate true inflow
        Zsim[i] = rho*(Zsim[i-1]) + ss.norm.rvs(0,1,1)*np.sqrt(1-rho**2)
        Qsim[i] = int(np.exp(Zsim[i]*logSd[i] + logAvg[i]))
        print("Inflow: " + str(Qsim[i]))
        print("Release: " + str(min(R[i], S[i] + Qsim[i])))
        
        # simulate next storage value
        S[i+1] = S[i] + Qsim[i] - min(R[i], S[i] + Qsim[i])
        
        if S[i+1] > K:
            print("Oh no! You overtopped the dam")
            break

print("Final reservoir storage: " + str(S[i+1]))
print("Reservoir capacity: " + str(K))

# update observations
Zfcst[i] = Zsim[i]
Qfcst[i] = Qsim[i]
Qfcst025[i] = Qsim[i]
Qfcst975[i] = Qsim[i]
plotForecast(i+1, S, K, Qfcst, Qfcst025, Qfcst975, Qsim)